In [2]:
import pandas as pd
from nsepython import *
import numpy as np
from nsepy import *
from datetime import datetime, timedelta

In [3]:
# Fetch data for Nifty 50
nifty_data = nsefetch('https://www.nseindia.com/api/equity-stockIndices?index=SECURITIES%20IN%20F%26O')

In [4]:
key_list = list(nifty_data.keys())

In [5]:
key_list

['name',
 'advance',
 'timestamp',
 'data',
 'metadata',
 'marketStatus',
 'date30dAgo',
 'date365dAgo']

In [6]:
df = pd.DataFrame(nifty_data['data'])
df = df.drop(0)

In [7]:
len(df)

222

In [8]:
df.columns

Index(['symbol', 'identifier', 'series', 'open', 'dayHigh', 'dayLow',
       'lastPrice', 'previousClose', 'change', 'pChange', 'totalTradedVolume',
       'totalTradedValue', 'lastUpdateTime', 'yearHigh', 'yearLow', 'nearWKH',
       'nearWKL', 'perChange365d', 'date365dAgo', 'chart365dPath',
       'date30dAgo', 'perChange30d', 'chart30dPath', 'chartTodayPath', 'meta'],
      dtype='object')

In [9]:
df_test = df[['symbol','open', 'dayHigh', 'dayLow','yearHigh', 'yearLow','lastPrice']]
df_test.head()

,symbol,open,dayHigh,dayLow,yearHigh,yearLow,lastPrice
1,LTIM,5882,6152.8,5882,6767.95,4513.55,6138.8
2,TECHM,1653.1,1710,1636.3,1807.7,1162.95,1708.7
3,IRCTC,772.8,800.75,755.3,1138.9,755.3,791.55
4,WIPRO,294.3,302.75,294.3,320,208.5,302.5
5,MGL,1295,1312,1240.05,1988,1075.25,1311


In [10]:
details_list = []
for i in range(1,182):
  x = df['meta'][i]
  del x['activeSeries']
  del x['debtSeries']
  del x['tempSuspendedSeries']
  del x['isFNOSec']
  del x['isCASec']
  del x['isSLBSec']
  del x['isDebtSec']
  del x['isSuspended']
  del x['isDelisted']
  del x['isETFSec']
  del x['quotepreopenstatus']
  del x['listingDate']
  #del x['slb_isin']
  del x['isMunicipalBond']

  details_list.append(pd.DataFrame([x]))

details_list[24]

,symbol,companyName,industry,isin,slb_isin
0,NESTLEIND,Nestle India Limited,Packaged Foods,INE239A01024,INE239A01024


In [11]:
len(details_list)

181

In [12]:
df_details = pd.concat(details_list[:], ignore_index=True)

In [13]:
df_details

,symbol,companyName,industry,isin,slb_isin
0,LTIM,LTIMindtree Limited,Computers - Software & Consulting,INE214T01019,INE214T01019
1,TECHM,Tech Mahindra Limited,Computers - Software & Consulting,INE669C01036,INE669C01036
2,IRCTC,Indian Railway Catering And Tourism Corporatio...,Tour Travel Related Services,INE335Y01020,INE335Y01020
3,WIPRO,Wipro Limited,Computers - Software & Consulting,INE075A01022,INE075A01022
4,MGL,Mahanagar Gas Limited,LPG/CNG/PNG/LNG Supplier,INE002S01010,INE002S01010
...,...,...,...,...,...
176,ABBOTINDIA,Abbott India Limited,Pharmaceuticals,INE358A01014,INE358A01014
177,ASHOKLEY,Ashok Leyland Limited,Commercial Vehicles,INE208A01029,INE208A01029
178,NTPC,NTPC Limited,Power Generation,INE733E01010,INE733E01010
179,SIEMENS,Siemens Limited,Heavy Electrical Equipment,INE003A01024,INE003A01024


In [14]:
df_details = df_details.drop(columns=['slb_isin'])

In [15]:
df_details[df_details.isnull().any(axis=1)]

,symbol,companyName,industry,isin


In [16]:
nse_df = pd.concat([df_test.set_index('symbol'), df_details.set_index('symbol')], axis = 1, join='outer').reset_index()

In [17]:
nse_df.columns

Index(['symbol', 'open', 'dayHigh', 'dayLow', 'yearHigh', 'yearLow',
       'lastPrice', 'companyName', 'industry', 'isin'],
      dtype='object')

In [18]:
nse_df = nse_df[['symbol','companyName', 'open', 'dayHigh', 'dayLow', 'yearHigh', 'yearLow', 'lastPrice', 'industry', 'isin']]

In [19]:
nse_df.rename(columns={'yearHigh':'52w_High', 'yearLow':'52w_Low', 'lastPrice':'Current_price'}, inplace = True)

In [20]:
nse_df['symbol'][0]

'LTIM'

In [21]:
keep_values = nse_df['symbol']
keep_values = keep_values.tolist()

In [22]:
import pytz
time_zone = pytz.timezone('Asia/Kolkata')
current_time = datetime.now().astimezone(time_zone)
current_time = current_time.strftime('%H:%M')
current_time = current_time.split(':')
current_time = [int(x) for x in current_time]
print(current_time)

[12, 0]


In [23]:
current_time[1]

0

In [24]:
current_date = datetime.now().date()
day_of_week = current_date.weekday()
if day_of_week == 5:
   current_date = current_date - timedelta(days = 1)
   print('date updated because it is saturday')

if (current_time[0] < 18):
    current_date = current_date - timedelta(days = 1)
    print('date updated because time is less than 6 pm')
    day_of_week = current_date.weekday()
    if day_of_week == 5:
     current_date = current_date - timedelta(days = 1)
     print('date updated because it is saturday')



current_date = current_date.strftime("%d-%m-%Y")
print(current_date)

date updated because time is less than 6 pm
09-01-2025


In [25]:
y = get_bhavcopy(current_date)
y = y[y['SYMBOL'].isin(keep_values)]
y.head()

,SYMBOL,SERIES,DATE1,PREV_CLOSE,OPEN_PRICE,HIGH_PRICE,LOW_PRICE,LAST_PRICE,CLOSE_PRICE,AVG_PRICE,TTL_TRD_QNTY,TURNOVER_LACS,NO_OF_TRADES,DELIV_QTY,DELIV_PER
63,AARTIIND,EQ,09-Jan-2025,397.85,398.55,417.35,397.85,414.95,413.40,409.49,7735966,31678.33,113688,1110557,14.36
71,ABB,EQ,09-Jan-2025,6657.00,6638.00,6692.00,6457.50,6483.00,6482.45,6574.51,193962,12752.05,33048,73754,38.02
72,ABBOTINDIA,EQ,09-Jan-2025,29608.25,29500.00,29930.00,29449.95,29700.00,29753.85,29743.09,7682,2284.86,4016,2641,34.38
73,ABCAPITAL,EQ,09-Jan-2025,178.91,179.00,179.03,173.00,174.29,173.52,175.15,2507589,4392.12,36301,1007778,40.19
76,ABFRL,EQ,09-Jan-2025,267.35,266.10,278.50,265.30,277.50,276.55,273.45,3896302,10654.25,26818,1579120,40.53


In [26]:
series_list = [' EQ']
y = y[y[' SERIES'].isin(series_list)]
len(y)

222

In [27]:
y = y[['SYMBOL', ' DELIV_QTY']]
y.head()

,SYMBOL,DELIV_QTY
63,AARTIIND,1110557
71,ABB,73754
72,ABBOTINDIA,2641
73,ABCAPITAL,1007778
76,ABFRL,1579120


In [28]:
nse_df = pd.concat([nse_df.set_index('symbol'), y.set_index('SYMBOL')], axis = 1, join='outer').reset_index(names = 'symbol')

In [29]:
losing_df = get_bhavcopy(current_date)
losing_df = losing_df[losing_df['SYMBOL'].isin(keep_values)]
losing_df = losing_df[losing_df[' SERIES'].isin([' EQ'])]
account_losing_days = losing_df[['SYMBOL']]

account_losing_days['LOSING_DAYS'] = 0
print(len(account_losing_days))
account_losing_days.head()

222


C:\Users\omshe\AppData\Local\Temp\ipykernel_17104\3518630890.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  account_losing_days['LOSING_DAYS'] = 0


,SYMBOL,LOSING_DAYS
63,AARTIIND,0
71,ABB,0
72,ABBOTINDIA,0
73,ABCAPITAL,0
76,ABFRL,0


In [30]:
# Get the current date
current_date = datetime.now().date()

# Define the number of days ago
days_ago = 34

# Calculate the date 20 days ago
date = current_date - timedelta(days=days_ago)

final_date = current_date

while(date != final_date):
  day_of_week = date.weekday()
  if day_of_week == 5:
    date = date + timedelta(days = 1)
    continue
  t = date
  t = t.strftime("%d-%m-%Y")


  # Wrap the get_bhavcopy call in a try-except block to handle HTTPError
  try:
    losing_df = get_bhavcopy(t)
  except urllib.error.HTTPError as e:
    print(f"Error fetching data for {t}: as NSE was closed on {date-timedelta(1)}")  # Print error message
    date = date + timedelta(days=1)  # Move to the next date
    continue  # Skip to the next iteration of the loop

  losing_df = losing_df[losing_df['SYMBOL'].isin(keep_values)]
  losing_df = losing_df[losing_df[' SERIES'].isin([' EQ'])]
  for index, row in losing_df.iterrows():
    if row[' OPEN_PRICE'] > row[' CLOSE_PRICE']:
        x = row['SYMBOL']
        index = account_losing_days.index[account_losing_days['SYMBOL'] == x].tolist()[0]
        account_losing_days.at[index, 'LOSING_DAYS'] += 1


  date = date + timedelta(days = 1)

In [31]:
account_losing_days.head()

,SYMBOL,LOSING_DAYS
63,AARTIIND,20
71,ABB,19
72,ABBOTINDIA,17
73,ABCAPITAL,21
76,ABFRL,21


In [32]:
nse_df = pd.concat([nse_df.set_index('symbol'), account_losing_days.set_index('SYMBOL')], axis = 1, join='outer').reset_index(names = 'symbol')

In [33]:
nse_df['52w_High'] = nse_df['52w_High'].astype(float)
nse_df['52w_Low'] = nse_df['52w_Low'].astype(float)
nse_df['Current_price'] = nse_df['Current_price'].astype(float)

In [34]:
nse_df['going_towards'] = np.nan
for index, row in nse_df.iterrows():
  high_diff = row['52w_High'] - row['Current_price']
  low_diff = row['Current_price'] - row['52w_Low']

  if (high_diff > low_diff):
    nse_df.at[index, 'going_towards'] = "Low"
  else:
    nse_df.at[index, 'going_towards'] = "High"

C:\Users\omshe\AppData\Local\Temp\ipykernel_17104\3158549882.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'High' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  nse_df.at[index, 'going_towards'] = "High"


In [35]:
nse_df['open'] = nse_df['open'].astype(float)
nse_df['Current_price'] = nse_df['Current_price'].astype(float)
nse_df['dayHigh'] = nse_df['dayHigh'].astype(float)
nse_df['dayLow'] = nse_df['dayLow'].astype(float)
nse_df['52w_High'] = nse_df['52w_High'].astype(float)
nse_df['52w_Low'] = nse_df['52w_Low'].astype(float)

In [36]:
nse_df['change%'] = np.nan
for index, row in nse_df.iterrows():
  change_pc = ((row['Current_price']-row['open'])/row['open'])*100
  change_pc = format(change_pc, ".2f")
  nse_df.at[index, 'change%'] = change_pc

C:\Users\omshe\AppData\Local\Temp\ipykernel_17104\1758213310.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4.37' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  nse_df.at[index, 'change%'] = change_pc


In [37]:
import yfinance as yf

def profit_sales_analysis(ticker, parameter):
    cases = ['000', '001', '010', '011', '100', '101', '110', '111']

    try:
        ticker = f"{ticker}.NS"
        ticker_obj = yf.Ticker(ticker)
        financials = ticker_obj.financials

        y_4 = financials.iloc[financials.index.get_loc(parameter), 0]
        y_3 = financials.iloc[financials.index.get_loc(parameter), 1]
        y_2 = financials.iloc[financials.index.get_loc(parameter), 2]
        y_1 = financials.iloc[financials.index.get_loc(parameter), 3]

        result = ""

        if (y_4 - y_3) > 0:
            result += "1"
        else:
            result += "0"
        
        if (y_3 - y_2) > 0:
            result += "1"
        else:
            result += "0"
            
        if (y_2 - y_1) > 0:
            result += "1"
        else:
            result += "0"
        
        if result == cases[0]:
            return "Not Recommended"
        elif result == cases[1]:
            return "Check quarterly results"
        elif result == cases[2]:
            return "Caution need monitoring"
        elif result == cases[3]:
            return "Recommended"
        elif result == cases[4]:
            return "Not Recommended"
        elif result == cases[5]:
            return "Caution need Monitoring"
        elif result == cases[6]:
            return "Caution need Monitoring"
        elif result == cases[7]:
            return "Highly Recommended"


    except Exception as e:
        #print(f"Error fetching profit/sales data for {ticker}: {e}")
        return np.nan  # Return NaN if any error occurs

In [38]:
import yfinance as yf

def debt_analysis(ticker, parameter):
    cases = ['000', '001', '010', '011', '100', '101', '110', '111']
    
    try:
        ticker = f"{ticker}.NS"
        ticker_obj = yf.Ticker(ticker)
        balance_sheet = ticker_obj.balance_sheet

        y_4 = balance_sheet.iloc[balance_sheet.index.get_loc(parameter), 0]
        y_3 = balance_sheet.iloc[balance_sheet.index.get_loc(parameter), 1]
        y_2 = balance_sheet.iloc[balance_sheet.index.get_loc(parameter), 2]
        y_1 = balance_sheet.iloc[balance_sheet.index.get_loc(parameter), 3]

        result = ""

        if (y_4 - y_3) > 0:
            result += "1"
        else:
            result += "0"
        
        if (y_3 - y_2) > 0:
            result += "1"
        else:
            result += "0"
            
        if (y_2 - y_1) > 0:
            result += "1"
        else:
            result += "0"
        
        if result == cases[0]:
            return "Highly Recommended"
        elif result == cases[1]:
            return "Caution need monitoring"
        elif result == cases[2]:
            return "Caution need monitoring"
        elif result == cases[3]:
            return "Not Recommended"
        elif result == cases[4]:
            return "Recommended"
        elif result == cases[5]:
            return "Caution need monitoring"
        elif result == cases[6]:
            return "Caution need Monitoring"
        elif result == cases[7]:
            return "Not Recommended"


    except Exception as e:
        #print(f"Error fetching Debt data for {ticker}: {e}")
        return np.nan  # Return NaN if any error occurs

In [39]:
x = np.nan
print(np.isnan(x))

True


In [40]:
symbol_list = (nse_df['symbol']).tolist()
symbol_list

['LTIM',
 'TECHM',
 'IRCTC',
 'WIPRO',
 'MGL',
 'PERSISTENT',
 'LTTS',
 'INFY',
 'HCLTECH',
 'MPHASIS',
 'BSOFT',
 'COFORGE',
 'NATIONALUM',
 'BATAINDIA',
 'HINDPETRO',
 'HINDCOPPER',
 'AUBANK',
 'MARICO',
 'MANAPPURAM',
 'TATACOMM',
 'ASTRAL',
 'ONGC',
 'DELHIVERY',
 'BAJAJFINSV',
 'NESTLEIND',
 'JSWSTEEL',
 'ESCORTS',
 'DIXON',
 'BAJFINANCE',
 'INDIGO',
 'SBILIFE',
 'BRITANNIA',
 'ATGL',
 'TATACONSUM',
 'NYKAA',
 'LICI',
 'BHARTIARTL',
 'IGL',
 'KEI',
 'ZOMATO',
 'BPCL',
 'LT',
 'ICICIPRULI',
 'UPL',
 'SONACOMS',
 'SUNTV',
 'ABB',
 'TATAMOTORS',
 'SAIL',
 'HDFCBANK',
 'ANGELONE',
 'MUTHOOTFIN',
 'OIL',
 'HDFCLIFE',
 'HINDUNILVR',
 'IOC',
 'INDIAMART',
 'NHPC',
 'COALINDIA',
 'JSL',
 'ABFRL',
 'ZYDUSLIFE',
 'PETRONET',
 'PIIND',
 'DABUR',
 'ADANIPORTS',
 'HEROMOTOCO',
 'GODREJCP',
 'SBICARD',
 'GMRAIRPORT',
 'NAUKRI',
 'SHREECEM',
 'TORNTPHARM',
 'ALKEM',
 'NCC',
 'M&M',
 'CIPLA',
 'TITAN',
 'SYNGENE',
 'POWERGRID',
 'RELIANCE',
 'IEX',
 'MFSL',
 'CESC',
 'BSE',
 'ADANIENSOL',
 'APOLL

In [41]:
symbol_list = (nse_df['symbol']).tolist()

Profit_Analysis = []
Sales_Analysis = []
Debt_Analysis = []

for symbol in symbol_list:

    profit = profit_sales_analysis(symbol, "EBIT")
    
    if not (isinstance(profit, str)):
        profit = profit_sales_analysis(symbol, "Net Income")
        print(f"Profit Value Changed for: {symbol}")
    
    sales = profit_sales_analysis(symbol, "Total Revenue")
    
    debt = debt_analysis(symbol, "Total Debt")
    
    Profit_Analysis.append(profit)
    Sales_Analysis.append(sales)
    Debt_Analysis.append(debt)

nse_df["Profit Analysis"] = Profit_Analysis
nse_df["Sales Analysis"] = Sales_Analysis
nse_df["Debt Analysis"] = Debt_Analysis
nse_df.head()

Profit Value Changed for: AUBANK
Profit Value Changed for: MANAPPURAM
Profit Value Changed for: NESTLEIND
Profit Value Changed for: BAJFINANCE
Profit Value Changed for: HDFCBANK
Profit Value Changed for: MUTHOOTFIN
Profit Value Changed for: SBICARD
Profit Value Changed for: ICICIBANK
Profit Value Changed for: KOTAKBANK
Profit Value Changed for: CANFINHOME
Profit Value Changed for: RBLBANK
Profit Value Changed for: M&MFIN
Profit Value Changed for: CANBK
Profit Value Changed for: CUB
Profit Value Changed for: BANKINDIA
Profit Value Changed for: BANKBARODA
Profit Value Changed for: IDFCFIRSTB
Profit Value Changed for: AXISBANK
Profit Value Changed for: LICHSGFIN
Profit Value Changed for: SBIN
Profit Value Changed for: YESBANK
Profit Value Changed for: AMBUJACEM
Profit Value Changed for: FEDERALBNK
Profit Value Changed for: PNB
Profit Value Changed for: INDIANB
Profit Value Changed for: LTF
Profit Value Changed for: PFC
Profit Value Changed for: BANDHANBNK
Profit Value Changed for: HUDCO
P

,symbol,companyName,open,dayHigh,dayLow,52w_High,52w_Low,Current_price,industry,isin,DELIV_QTY,LOSING_DAYS,going_towards,change%,Profit Analysis,Sales Analysis,Debt Analysis
0,LTIM,LTIMindtree Limited,5882.0,6152.80,5882.00,6767.95,4513.55,6138.80,Computers - Software & Consulting,INE214T01019,97126,18,High,4.37,Highly Recommended,Highly Recommended,Not Recommended
1,TECHM,Tech Mahindra Limited,1653.1,1710.00,1636.30,1807.70,1162.95,1708.70,Computers - Software & Consulting,INE669C01036,365123,18,High,3.36,Check quarterly results,Recommended,Caution need monitoring
2,IRCTC,Indian Railway Catering And Tourism Corporatio...,772.8,800.75,755.30,1138.90,755.30,791.55,Tour Travel Related Services,INE335Y01020,291757,12,Low,2.43,Highly Recommended,Highly Recommended,Caution need monitoring
3,WIPRO,Wipro Limited,294.3,302.75,294.30,320.00,208.50,302.50,Computers - Software & Consulting,INE075A01022,3629759,14,High,2.79,Highly Recommended,Recommended,Caution need monitoring
4,MGL,Mahanagar Gas Limited,1295.0,1312.00,1240.05,1988.00,1075.25,1311.00,LPG/CNG/PNG/LNG Supplier,INE002S01010,397670,11,Low,1.24,Caution need Monitoring,Recommended,Not Recommended


In [42]:
def get_company_details(ticker, detail):
    t = f"{ticker}.NS"
    t = yf.Ticker(t)
    return t.info[detail]

In [43]:
for index in range(181, len(nse_df)):
    if pd.isna(nse_df.at[index, "companyName"]):
        symbol = nse_df.at[index, "symbol"]
                
        # Try to get the company name (longName, fallback to shortName)
        try:
            name = get_company_details(symbol, "longName")
        except Exception as e:
            print(f"Error getting longName for symbol {symbol}: {e}")
            try:
                name = get_company_details(symbol, "shortName")
            except Exception as e:
                print(f"Error getting shortName for symbol {symbol}: {e}")
                name = None  # Use None or a default value if both fail
        
        # Get industry information
        try:
            industry = get_company_details(symbol, "industry")
        except Exception as e:
            print(f"Error getting industry for symbol {symbol}: {e}")
            industry = None  # Use None or a default value if this fails
        
        # Update the DataFrame
        nse_df.at[index, "companyName"] = name
        nse_df.at[index, "industry"] = industry


In [44]:
nse_df['Profit Analysis'].isna().sum()

np.int64(2)

### News Scrapping and analysis


#### Performed in Colab

In [45]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.options import Options
import time

def scrape_yahoo_finance_news(symbol: str):
    
    print(symbol)
    # Initialize Selenium WebDriver
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    
    driver = webdriver.Chrome(options=chrome_options)
    
    # Construct the URL dynamically
    url = f"https://finance.yahoo.com/quote/{symbol}.NS/news/"
    
    try:
        # Navigate to the URL
        driver.get(url)
        wait = WebDriverWait(driver, 1)
        
        # Check if the news container exists
        
        try:
            news_container = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-testid="news-tabs-container"]')))
        except TimeoutException:
            return f"{symbol} - No news"
        
        
        # Locate the list of news items
        try:
            news_list = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.stream-items.yf-1usaaz9')))
            news_items = news_list.find_elements(By.CSS_SELECTOR, 'li.stream-item.story-item.yf-1usaaz9')
            
            titles = []
            descriptions = []
            
            for item in news_items:
                try:
                    # Extract title and description
                    title_tag = item.find_element(By.CSS_SELECTOR, 'a.subtle-link.fin-size-small.titles.noUnderline.yf-1xqzjha h3')
                    description_tag = item.find_element(By.CSS_SELECTOR, 'a.subtle-link.fin-size-small.titles.noUnderline.yf-1xqzjha p.clamp.yf-18q3fnf')
                    titles.append(title_tag.text)
                    descriptions.append(description_tag.text)
                except NoSuchElementException:
                    continue
            
            # Return the titles and descriptions if any
            return titles, descriptions if titles and descriptions else f"{symbol} - No news"
        
        except TimeoutException:
            return f"{symbol} - No news"
    
    finally:
        # Close the WebDriver
        driver.quit()



In [46]:
from textblob import TextBlob

def news_analysis(news):
    
    # Check the input type
    if isinstance(news, str):
        return "NEUTRAL"

    if not isinstance(news, tuple) or len(news) != 2:
        raise ValueError("Input must be a tuple with two elements: (titles, descriptions).")

    titles, descriptions = news

    if not (isinstance(titles, list) and isinstance(descriptions, list)):
        raise ValueError("Both titles and descriptions must be lists.")

    if len(titles) != len(descriptions):
        raise ValueError("Titles and descriptions lists must have the same length.")

    sentiment_record = {"positive": 0, "negative": 0, "neutral": 0}

    # Perform sentiment analysis
    for title, description in zip(titles, descriptions):
        text = f"{title} {description}"
        analysis = TextBlob(text).sentiment.polarity

        if analysis > 0:
            sentiment_record["positive"] += 1
        elif analysis < 0:
            sentiment_record["negative"] += 1
        else:
            sentiment_record["neutral"] += 1

    print(sentiment_record)

    # Determine final sentiment
    if sentiment_record["positive"] > max(sentiment_record["negative"], sentiment_record["neutral"]):
        return "POSITIVE"
    elif sentiment_record["negative"] > max(sentiment_record["positive"], sentiment_record["neutral"]):
        return "NEGATIVE"
    else:
        return "NEUTRAL"




In [47]:
symbol_list = (nse_df['symbol']).tolist()

In [48]:
NEWS = []

for symbol in symbol_list:
    news = scrape_yahoo_finance_news(symbol)
    result = news_analysis(news)
    NEWS.append(result)

LTIM
{'positive': 9, 'negative': 4, 'neutral': 7}
TECHM
{'positive': 15, 'negative': 4, 'neutral': 1}
IRCTC
WIPRO
{'positive': 8, 'negative': 6, 'neutral': 6}
MGL
{'positive': 1, 'negative': 0, 'neutral': 0}
PERSISTENT
{'positive': 15, 'negative': 5, 'neutral': 0}
LTTS
{'positive': 6, 'negative': 6, 'neutral': 8}
INFY
{'positive': 7, 'negative': 5, 'neutral': 8}
HCLTECH
{'positive': 19, 'negative': 1, 'neutral': 0}
MPHASIS
{'positive': 2, 'negative': 0, 'neutral': 0}
BSOFT
{'positive': 1, 'negative': 0, 'neutral': 0}
COFORGE
{'positive': 13, 'negative': 2, 'neutral': 0}
NATIONALUM
{'positive': 1, 'negative': 0, 'neutral': 0}
BATAINDIA
HINDPETRO
{'positive': 2, 'negative': 2, 'neutral': 1}
HINDCOPPER
AUBANK


SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location C:\Program Files\Google\Chrome\Application\chrome.exe is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
	GetHandleVerifier [0x00007FF735CB80D5+2992373]
	(No symbol) [0x00007FF73594BFD0]
	(No symbol) [0x00007FF7357E590A]
	(No symbol) [0x00007FF7358201C0]
	(No symbol) [0x00007FF73581BC39]
	(No symbol) [0x00007FF73586A6AC]
	(No symbol) [0x00007FF735869C90]
	(No symbol) [0x00007FF73585F113]
	(No symbol) [0x00007FF73582A918]
	(No symbol) [0x00007FF73582BA81]
	GetHandleVerifier [0x00007FF735D16A2D+3379789]
	GetHandleVerifier [0x00007FF735D2C32D+3468109]
	GetHandleVerifier [0x00007FF735D20043+3418211]
	GetHandleVerifier [0x00007FF735AAC78B+847787]
	(No symbol) [0x00007FF73595757F]
	(No symbol) [0x00007FF735952FC4]
	(No symbol) [0x00007FF73595315D]
	(No 

#### Continue Here

In [ ]:
nse_df["News"] = NEWS
nse_df.head()

,symbol,companyName,open,dayHigh,dayLow,52w_High,52w_Low,Current_price,industry,isin,DELIV_QTY,LOSING_DAYS,going_towards,change%,Profit Analysis,Sales Analysis,Debt Analysis,News
0,INDIAMART,Indiamart Intermesh Limited,2218.00,2312.70,2188.90,3198.40,2165.55,2305.00,Internet & Catalogue Retail,INE933S01016,242006,17,Low,3.92,Not Recommended,Highly Recommended,Highly Recommended,NEUTRAL
1,BSE,BSE Limited,5090.00,5465.00,5090.00,5837.95,1941.05,5360.50,Exchange and Data Platform,INE118H01025,402345,13,High,5.31,Caution need Monitoring,Highly Recommended,Highly Recommended,POSITIVE
2,NCC,NCC Limited,262.00,274.95,262.00,364.50,171.55,273.30,Civil Construction,INE868B01028,1628416,17,High,4.31,Highly Recommended,Highly Recommended,Recommended,POSITIVE
3,CHAMBLFERT,Chambal Fertilizers & Chemicals Limited,480.95,505.50,480.65,574.35,332.05,502.55,Fertilizers,INE085A01013,633723,22,High,4.49,Not Recommended,Recommended,Caution need monitoring,NEUTRAL
4,CYIENT,Cyient Limited,1751.00,1837.70,1749.15,2365.00,1651.50,1831.20,IT Enabled Services,INE136B01020,223347,18,Low,4.58,Highly Recommended,Highly Recommended,Caution need monitoring,NEUTRAL


In [ ]:
nse_df.head()

NameError: name 'nse_df' is not defined

In [ ]:
# Downloading the above data as excel sheet

file_path = " Today's_data.xlsx"
date = datetime.now().date()
date = date.strftime("%d-%m-%Y")
file_path = date + file_path
nse_df.to_excel(file_path, index = False)
print("File downloaded")

File downloaded
